In [1]:
# Installation simple
!pip install -q sentence-transformers datasets accelerate huggingface_hub

# Login HuggingFace
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login

secrets = UserSecretsClient()
hf_token = secrets.get_secret("HF_TOKEN")
login(token=hf_token)
print("OK")

OK


In [2]:
# Télécharger les données d'entraînement
!wget -q https://raw.githubusercontent.com/pierrealexandreguillemin-a11y/pocket_arbiter/main/data/training/triplets_training.jsonl

import json
triplets = [json.loads(l) for l in open("triplets_training.jsonl") if l.strip()]
print(f"Triplets chargés: {len(triplets)}")

Triplets chargés: 2152


In [3]:
import torch
import gc
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, SentenceTransformerTrainingArguments
from sentence_transformers.losses import MultipleNegativesRankingLoss
from datasets import Dataset

# Nettoyer la mémoire
gc.collect()
torch.cuda.empty_cache()

# Info GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    vram = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"VRAM: {vram:.1f} GB")

# Charger EmbeddingGemma
model = SentenceTransformer("google/embeddinggemma-300M", device=device)
print(f"Model loaded: {model.get_sentence_embedding_dimension()} dims")

# Gradient checkpointing
model[0].auto_model.gradient_checkpointing_enable()
print("Gradient checkpointing enabled")

2026-01-17 10:05:32.055668: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768644332.249513      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768644332.306613      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768644332.750592      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768644332.750623      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768644332.750625      24 computation_placer.cc:177] computation placer alr

Device: cuda
GPU: Tesla T4
VRAM: 15.8 GB


modules.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/997 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/18.7k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/9.44M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

3_Dense/model.safetensors:   0%|          | 0.00/9.44M [00:00<?, ?B/s]

Model loaded: 768 dims
Gradient checkpointing enabled


In [4]:
# Configuration optimisée pour Kaggle GPU (P100/T4)
# Kaggle a plus de RAM système donc plus de marge

trainer = SentenceTransformerTrainer(
    model=model,
    args=SentenceTransformerTrainingArguments(
        output_dir="embeddinggemma-chess-arbiter-fr",
        num_train_epochs=3,
        per_device_train_batch_size=2,            # Kaggle peut gérer 2
        gradient_accumulation_steps=8,            # Effective batch = 16
        learning_rate=2e-5,
        warmup_ratio=0.1,
        fp16=False,                               # EmbeddingGemma ne supporte pas fp16
        bf16=False,
        logging_steps=50,
        save_strategy="epoch",
        report_to="none",
        dataloader_drop_last=True,
        optim="adamw_torch_fused",
        dataloader_num_workers=2,                 # Kaggle a plus de CPU
    ),
    train_dataset=Dataset.from_list(triplets),
    loss=MultipleNegativesRankingLoss(model)
)

print("Démarrage training...")
print(f"Steps par epoch: {len(triplets) // 2 // 8}")
print(f"Total steps: {len(triplets) // 2 // 8 * 3}")

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Démarrage training...
Steps par epoch: 134
Total steps: 402


In [5]:
# TRAINING
trainer.train()
model.save("embeddinggemma-chess-arbiter-fr")
print("Training terminé!")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
50,1.085200
100,0.455000
150,0.396100
200,0.297800


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Training terminé!


In [6]:
# Évaluation rapide (ISO 42001)
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
import random

finetuned = SentenceTransformer("embeddinggemma-chess-arbiter-fr")
test_samples = random.sample(triplets, min(10, len(triplets)))

print("=" * 60)
print("ÉVALUATION QUALITÉ (ISO 42001 - Recall)")
print("=" * 60)

correct = 0
for i, t in enumerate(test_samples):
    q_emb = finetuned.encode(t["anchor"])
    pos_emb = finetuned.encode(t["positive"])
    neg_emb = finetuned.encode(t["negative"])
    
    sim_pos = cos_sim(q_emb, pos_emb).item()
    sim_neg = cos_sim(q_emb, neg_emb).item()
    
    is_correct = sim_pos > sim_neg
    correct += int(is_correct)
    
    status = "OK" if is_correct else "FAIL"
    print(f"[{i+1}] {status} | pos={sim_pos:.3f} neg={sim_neg:.3f}")

accuracy = correct / len(test_samples) * 100
print("=" * 60)
print(f"RÉSULTAT: {correct}/{len(test_samples)} = {accuracy:.0f}%")
print(f"Cible: >=80% → {'PASS' if accuracy >= 80 else 'FAIL'}")

ÉVALUATION QUALITÉ (ISO 42001 - Recall)
[1] OK | pos=0.850 neg=0.263
[2] OK | pos=0.839 neg=0.248
[3] OK | pos=0.800 neg=0.248
[4] OK | pos=0.798 neg=0.263
[5] OK | pos=0.770 neg=0.382
[6] OK | pos=0.653 neg=0.055
[7] OK | pos=0.647 neg=0.081
[8] OK | pos=0.908 neg=0.459
[9] OK | pos=0.799 neg=0.197
[10] OK | pos=0.734 neg=0.363
RÉSULTAT: 10/10 = 100%
Cible: >=80% → PASS


In [7]:
# Télécharger le modèle fine-tuné
import shutil
shutil.make_archive("embeddinggemma-chess-arbiter-fr", "zip", "embeddinggemma-chess-arbiter-fr")

# Sur Kaggle: le fichier sera dans /kaggle/working/
print("Modèle exporté: embeddinggemma-chess-arbiter-fr.zip")
print("Télécharge depuis l'onglet 'Output' à droite →")

Modèle exporté: embeddinggemma-chess-arbiter-fr.zip
Télécharge depuis l'onglet 'Output' à droite →
